In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
!pip install -q openai-whisper transformers sentencepiece torch ffmpeg-python


In [ ]:
import whisper

# Load Whisper model
model = whisper.load_model("base")

# Get uploaded audio file name
audio_file = list(uploaded.keys())[0]

print("Processing audio:", audio_file)

# Transcribe audio
result = model.transcribe(audio_file)

lecture_text = result["text"]

print("\n===== LECTURE TEXT =====\n")
print(lecture_text)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Prepare input for summarization
input_text = "summarize: " + lecture_text
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    max_length=512,
    truncation=True
)

# Generate summary
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=150,
    min_length=60,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

notes = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("===== GENERATED STUDY NOTES =====\n")
print(notes)


In [ ]:
def generate_quiz(text):
    sentences = text.split(".")
    questions = []

    for s in sentences:
        s = s.strip()
        if len(s) > 25:
            questions.append("Explain: " + s + "?")

    return questions[:5]   # limit to 5 questions


quiz = generate_quiz(notes)

print("===== QUIZ QUESTIONS =====\n")
for i, q in enumerate(quiz, 1):
    print(f"{i}. {q}")


In [ ]:
def generate_flashcards(text):
    sentences = text.split(".")
    flashcards = []

    for s in sentences:
        s = s.strip()
        if len(s) > 20:
            flashcards.append({
                "Question": "Explain briefly:",
                "Answer": s
            })

    return flashcards


flashcards = generate_flashcards(notes)

print("===== FLASHCARDS =====\n")
for i, card in enumerate(flashcards, 1):
    print(f"Flashcard {i}")
    print("Q:", card["Question"])
    print("A:", card["Answer"])
    print()


In [ ]:
from google.colab import files

with open("Lecture_Output.txt", "w") as f:
    f.write("===== LECTURE STUDY NOTES =====\n\n")
    f.write(notes + "\n\n")

    f.write("===== QUIZ QUESTIONS =====\n\n")
    for i, q in enumerate(quiz, 1):
        f.write(f"{i}. {q}\n")

    f.write("\n===== FLASHCARDS =====\n\n")
    for i, card in enumerate(flashcards, 1):
        f.write(f"Flashcard {i}\n")
        f.write("Q: " + card["Question"] + "\n")
        f.write("A: " + card["Answer"] + "\n\n")

files.download("Lecture_Output.txt")


In [ ]:
!pip install fpdf


In [ ]:
!pip install reportlab


In [ ]:
!pip install fpdf2


In [ ]:
!pip install reportlab


In [ ]:
!pip install reportlab


In [ ]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm

# Save PDF in Colab working directory
pdf_path = "/content/Lecture_Voice_to_Notes_Report.pdf"

# Create PDF document
doc = SimpleDocTemplate(pdf_path, pagesize=A4, rightMargin=2*cm, leftMargin=2*cm, topMargin=2*cm, bottomMargin=2*cm)
styles = getSampleStyleSheet()
story = []

# Title
story.append(Paragraph("Internship Project Report", styles['Title']))
story.append(Spacer(1, 12))

# Sections content
sections = {
    "Project Details": """Project Title: Lecture Voice-to-Notes Generator<br/>
Student Name: Aman Kumar<br/>
Department: [Your Department Name]<br/>
College/Institute: [Your College Name]<br/>
Internship Duration: [Start Date – End Date]<br/>
Supervisor Name: [Your Guide's Name]""",

    "1. Introduction": """In today’s academic environment, students often miss key points during lectures because it is difficult to listen and take notes simultaneously.
The Lecture Voice-to-Notes Generator automates this process by converting lecture audio into text, summarizing it into concise notes, and generating quizzes and flashcards for effective revision.<br/>
Objective:<br/>
- Convert lecture audio to text using AI-based speech-to-text models.<br/>
- Summarize lecture content into clear study notes.<br/>
- Generate quiz questions and flashcards automatically.<br/>
- Improve learning efficiency and student productivity.""",

    "2. Problem Statement": """Students frequently struggle with:<br/>
- Missing important points while taking notes.<br/>
- Spending excessive time writing long notes.<br/>
- Difficulty in revising lectures efficiently.<br/>
Need: A solution that automatically converts lecture speech into structured notes, quizzes, and flashcards.""",

    "3. Proposed Solution": """The project uses AI technologies:<br/>
1. Speech-to-Text (Whisper): Converts audio lectures into editable text.<br/>
2. Text Summarization (T5 Model): Converts long transcripts into concise notes.<br/>
3. Quiz & Flashcard Generator: Automatically creates questions and flashcards for revision.<br/>
System Workflow: Lecture Audio → Speech-to-Text → Text Summarization → Notes → Quiz/Flashcards → Output""",

    "4. Tools and Technologies Used": """Speech-to-Text: Whisper (OpenAI)<br/>
Text Summarization: T5 Transformer (HuggingFace)<br/>
Quiz & Flashcard Generator: Python custom logic<br/>
Programming Language: Python 3<br/>
Platform: Google Colab<br/>
File Handling: Google Colab Upload & Download""",

    "5. Implementation": """Step 1: Upload Audio<br/>
Step 2: Speech-to-Text Conversion<br/>
Step 3: Summarization<br/>
Step 4: Quiz Generation<br/>
Step 5: Flashcard Generation<br/>
Step 6: Output""",

    "6. Sample Output": """Lecture Notes:<br/>
Artificial Intelligence is a branch of computer science that focuses on creating intelligent machines capable of performing tasks that normally require human intelligence...<br/><br/>
Quiz Questions:<br/>1. Explain: Artificial Intelligence is a branch of computer science that focuses on creating intelligent machines?<br/><br/>
Flashcards:<br/>Q: Explain briefly:<br/>A: Artificial Intelligence is a branch of computer science that focuses on creating intelligent machines...""",

    "7. Advantages": """- Saves students’ time in note-taking.<br/>
- Ensures no important points are missed.<br/>
- Provides structured notes, quizzes, and flashcards automatically.<br/>
- Easy revision for exams and self-study.""",

    "8. Future Scope": """- Real-time lecture transcription.<br/>
- Multi-language support.<br/>
- Mobile app integration.<br/>
- Export to PDF/Word for easy sharing.<br/>
- Integration with Learning Management Systems (LMS).""",

    "9. Conclusion": """The Lecture Voice-to-Notes Generator automates lecture note-taking, summarization, and quiz/flashcard generation.<br/>
It enhances learning efficiency, ensures important points are captured, and provides a complete study tool for students.""",

    "10. References": """1. OpenAI Whisper Documentation: https://github.com/openai/whisper<br/>
2. HuggingFace Transformers: https://huggingface.co/transformers<br/>
3. Google Colab: https://colab.research.google.com<br/>
4. T5: Text-to-Text Transfer Transformer, Google Research"""
}

# Add sections to PDF
for title, content in sections.items():
    story.append(Paragraph(f"<b>{title}</b>", styles['Heading2']))
    story.append(Spacer(1, 6))
    story.append(Paragraph(content, styles['Normal']))
    story.append(Spacer(1, 12))

# Build PDF
doc.build(story)

print("PDF created successfully!")
print("Download path:", pdf_path)


In [ ]:
from google.colab import files
files.download("/content/Lecture_Voice_to_Notes_Report.pdf")


In [ ]:
!pip install python-pptx


In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt

# Create a presentation object
prs = Presentation()

# Slide layout: 0=Title Slide, 1=Title and Content
title_slide_layout = prs.slide_layouts[0]
content_slide_layout = prs.slide_layouts[1]

# Title slide
slide = prs.slides.add_slide(title_slide_layout)
slide.shapes.title.text = "Internship Project Report"
slide.placeholders[1].text = "Lecture Voice-to-Notes Generator\nStudent: Aman Kumar"

# Sections content
sections = {
    "Introduction": """Students often miss key points during lectures because it is difficult to listen and take notes simultaneously.
This project converts lecture audio into text, summarizes it into notes, and generates quizzes and flashcards.""",

    "Problem Statement": """- Missing important points while taking notes
- Spending excessive time writing long notes
- Difficulty in revising lectures efficiently""",

    "Proposed Solution": """- Speech-to-Text (Whisper) for transcription
- Text Summarization (T5) for notes
- Quiz & Flashcard Generator for revision""",

    "Tools & Technologies": """- Whisper (OpenAI)
- T5 Transformer (HuggingFace)
- Python 3, Google Colab""",

    "Implementation": """1. Upload Audio
2. Speech-to-Text Conversion
3. Summarization
4. Quiz Generation
5. Flashcard Generation
6. Output""",

    "Sample Output": """Lecture Notes, Quiz Questions, Flashcards (example content)""",

    "Advantages": """- Saves time
- Ensures no key points are missed
- Provides structured notes, quizzes, and flashcards
- Easy revision""",

    "Future Scope": """- Real-time transcription
- Multi-language support
- Mobile app integration
- Export to PDF/Word
- LMS integration""",

    "Conclusion": """The Lecture Voice-to-Notes Generator automates note-taking and revision, improving learning efficiency.""",

    "References": """1. OpenAI Whisper: https://github.com/openai/whisper
2. HuggingFace Transformers: https://huggingface.co/transformers
3. Google Colab: https://colab.research.google.com"""
}

# Add slides
for title, content in sections.items():
    slide = prs.slides.add_slide(content_slide_layout)
    slide.shapes.title.text = title
    slide.placeholders[1].text = content

# Save presentation
pptx_file = "/content/Lecture_Voice_to_Notes_Report.pptx"
prs.save(pptx_file)
pptx_file


In [ ]:
from google.colab import files
files.download("/content/Lecture_Voice_to_Notes_Report.pptx")
